[<img src="../header.svg">](../index.ipynb)

---
# Lösen der Poisson-Gleichung mit FEM in UG4 

Wir lösen in diesem Beispiel 
$$-\triangle u = f $$
auf dem Einheitsquadrat mit Dirichlet-Randwerten. 

Dazu verwenden wir hier einen entprechenden Jupyter-Kernel für die Software UG4, der LUA unterstützt. Eine Erweiterung folgt im [nachfolgenden Beispiel](./tutorial-fem-02.ipynb).

## Initialisierung 

In [1]:
InitUG(2, AlgebraType("CPU", 1));  -- Initialize world dimension dim=2 and default algebra type
ug_load_script("ug_util.lua")     -- Load utility scripts (e.g. from from ugcore/scripts)
ug_load_script("util/refinement_util.lua")

* Initializing: paths... done, bridge... done, plugins... done                 *
Loading Plugin Lua Content from /Users/anaegel/Software/ug4-git/plugins/Poroelasticity/lua/lua-include.lua
Loading Plugin Lua Content from /Users/anaegel/Software/ug4-git/plugins/d3f/lua/lua-include.lua
Loading Plugin Lua Content from /Users/anaegel/Software/ug4-git/plugins/Limex/lua/lua-include.lua
LIMEX-Plugin: Loading util.limex.*
Loading Plugin Lua Content from /Users/anaegel/Software/ug4-git/plugins/RepoTrend/lua/lua-include.lua
RepoTrend-Plugin: Extended package.path:./?.lua;/usr/local/share/lua/5.1/?.lua;/usr/local/share/lua/5.1/?/init.lua;/usr/local/lib/lua/5.1/?.lua;/usr/local/lib/lua/5.1/?/init.lua;/Users/anaegel/Software/ug4-git/plugins/RepoTrend/lua/?.lua


## Problemdefinition

Zur Problemdefinition definieren wir einige Variablen:

In [2]:
myGridName= "grids/unit_square_tri.ugx" --"grids/unit_square_tri.ugx",
myNumRefs= 3
mySubsets = {"Inner", "Boundary"}

Randbedingungen setzen wir über einen Callback:

In [3]:
-- Callback fuer Randbedingungen
function myDirichletBndCallback(x, y, t)
    if (y==1) then return true, 0.0 
    elseif (y==0) then  return true, math.sin(math.pi*1*x)
    else return false, 0.0 
    end
end

## Rechengebiet
- Einlesen aus einer Datei:

In [4]:
dom = Domain()
LoadDomain(dom, myGridName)

- Test: Sind alle Subsets vorhanden?

In [5]:
print(util.CheckSubsets(dom, mySubsets))

true


* Uniforme Verfeinerung

In [6]:
local refiner = GlobalDomainRefiner(dom)
for i=1,myNumRefs do
    write(i .. " ")
    refiner:refine()
end

1 2 3 

- Alternativ geht es auch ganz bequem in einer Zeile:

In [7]:
dom = util.CreateDomain(myGridName, myNumRefs, myRequiredSubsets)

Loading Domain grids/unit_square_tri.ugx ... done.
Performing integrity check on domain ... done.
Refining(3): 1 2 3 done.


## Ansatzraum
Erzeuge Ansatzraum mit stückweise linearen Funktionen (elementweise Polynome 1. Grades):

In [8]:
-- Setup for FEM approximation space.
approxSpace = ApproximationSpace(dom)
approxSpace:add_fct("u", "Lagrange", 1)  -- Linear ansatz functions

-- More inits.
approxSpace:init_levels()
approxSpace:init_top_surface()
approxSpace:print_statistic()

| ---------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                  |
|  Algebra: Block 1 (divide by 1 for #Index)                                   |
|                                                                              |
|    GridLevel   |       Domain |     0: Inner |  1: Boundary                  |
| ---------------------------------------------------------------------------- |
| (lev,    0)    |            9 |            1 |            8 |
| (lev,    1)    |           25 |            9 |           16 |
| (lev,    2)    |           81 |           49 |           32 |
| (lev,    3)    |          289 |          225 |           64 |
| (lev,    0, g) |            9 |            1 |            8 |
| (lev,    1, g) |           25 |            9 |           16 |
| (lev,    2, g) |           81 |           49 |           32 |
| (lev,    3, g) |          289 |          225 |           64 |
| 

## Diskretisierung

Erzeuge Objekt für eine **Elementdiskretisierung** für die Konvektions-Diffusionsgleichung.

In [9]:
elemDisc = ConvectionDiffusion("u", "Inner", "fe")
elemDisc:set_diffusion(1.0)

-- Optional: Setze rechte Seite f
-- elemDisc:set_source(1.0)

Erzeuge Objekt für **Randbedingungen**:

In [10]:
dirichletBND = DirichletBoundary()
dirichletBND:add("myDirichletBndCallback", "u", "Boundary")

Füge beides zu einer **Gebietsdiskretisierung** hinzu:

In [11]:
domainDisc = DomainDiscretization(approxSpace)
domainDisc:add(elemDisc)
domainDisc:add(dirichletBND)

## Konfiguration eines iterativen Lösers

Ein sog. Mehrgitterverfahren (=> Modellierung und Simulation 2) hat lediglich lineare Komplexität

In [12]:
-- MG solver setup (using 'util/solver_util.lua')
local solverDesc = {
    type = "bicgstab",
    precond = {
        type = "gmg",
        approxSpace = approxSpace,
        smoother = "sgs",
        baseSolver = "lu"
    }
}
solver = util.solver.CreateSolver(solverDesc)

## Assembliere und löse LGS

In [13]:
A = AssembledLinearOperator(domainDisc)
x = GridFunction(approxSpace)
b = GridFunction(approxSpace)
x:set(0.0)


domainDisc:assemble_linear(A, b)
domainDisc:adjust_solution(x)

solver:init(A, x)
solver:apply(x, b)


   % %%%%%%%%             BiCGStab              %%%%%%%%%%%
   % %%%%%%%%   (Precond: Geometric MultiGrid)  %%%%%%%%%%%
   %   Iter      Defect         Rate 
   %    0:    2.828427e+00      -------
   %    1:    3.551370e-02    1.255599e-02
   %    2:    4.792401e-04    1.349451e-02
   %    3:    8.145414e-06    1.699652e-02
   %    4:    2.002548e-07    2.458498e-02
   % Relative reduction 1.000000e-06 reached after 4 steps.
   % Average reduction over 4 steps: 1.631209e-02
   % %%%%%  Iteration converged  %%%%%



### Ausgabe als *.vtu bzw. *.vec-Datei
a) Für die Lösung

In [14]:
local solFileName = "x_solution"
WriteGridFunctionToVTK(x, solFileName)
SaveVectorForConnectionViewer(x, solFileName .. ".vec")

b) Für die rechte Seite $b$ und Matrix $A$

In [15]:
local matFileName = "A_matrix.mat"
SaveMatrixForConnectionViewer(x, A, matFileName)

local rhsFileName = "b_rhs"
SaveVectorForConnectionViewer(b, rhsFileName.. ".vec")